In [1]:
#This script opens the citation dataset and perform calculations in patent level
#Renato Kogeyama

#Miami, Jan 28, 2020
#Script to calculate citations received by a patent
#citation_id with less than 3 characters have no meaning
#they are causing problems moving forward
#so here I drop them


In [2]:
import pandas as pd
import numpy as np
import re
import glob

In [3]:
# %matplotlib inline

In [4]:
file_list=glob.glob("data/citation/*")
dfs = [pd.read_parquet(f, columns=['patent_id']) for f in file_list]
dst = 'data/cit_received.parquet.gz'

for i, df in enumerate(dfs):
    if i==0:
        output=df.groupby([df.index]).count() 
    else:
        output=pd.concat([output, df.groupby(df.index).count()]) # following Nemet and Johson, this variable is relevant for citations made


#cited patents registers total citations received 
# df=df.groupby(['citation_id']).count().iloc[:,0].reset_index() 
#Series, patent-level
# cited_patents.dropna(0, inplace=True) 
#Series, '0' implies that rows are dropped
#I should check this, because later I make citation back as index for merging purposes
# cited_patents=cited_patents.reset_index() #Dataframe")

output.rename(columns={'patent_id': 'cit_received'}, inplace=True)
# np.log(output['cit_received']).hist()

In [5]:
output=output.groupby(output.index).sum()

In [6]:
output.nlargest(15, 'cit_received')

,cit_received
citation_id,
4683202,4677
4683195,4083
5523520,4047
7674650,3498
5731856,3402
7061014,3391
7297977,3371
6294274,3359
6727522,3354


In [7]:
output.to_parquet(dst, compression='gzip')